In [13]:
!pip install azure-cosmos plotly


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from azure.cosmos import CosmosClient, exceptions
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime, timedelta

In [15]:
# Initialize Cosmos DB client
COSMOS_DB_URL = 'YOUR_COSMOS_DB_URL'
COSMOS_DB_KEY = 'YOUR_COSMOS_DB_KEY'

DATABASE_NAME = 'bus_data'
VEHICLE_CONTAINER_NAME = 'vehicle_data'
STOP_CONTAINER_NAME = 'stop_data'

# Initialize Cosmos client
client = CosmosClient(COSMOS_DB_URL, COSMOS_DB_KEY)
database = client.get_database_client(DATABASE_NAME)
vehicle_container = database.get_container_client(VEHICLE_CONTAINER_NAME)
stop_container = database.get_container_client(STOP_CONTAINER_NAME)

In [16]:
def get_vehicle_and_stop_data(timestamp):
    # Query for vehicle data based on the provided timestamp
    vehicle_query = """
    SELECT * FROM c 
    WHERE c["scheduled_task_time"] = @timestamp
    """
    vehicle_parameters = [{"name": "@timestamp", "value": timestamp}]
    
    try:
        # Get vehicle data
        vehicle_data = list(vehicle_container.query_items(
            query=vehicle_query,
            parameters=vehicle_parameters,
            enable_cross_partition_query=True
        ))

        if not vehicle_data:
            print("No vehicles found for the given timestamp.")
            return [], []

        # Extract route keys from the vehicle data
        route_keys = set([vehicle['route_key'] for vehicle in vehicle_data])

        # Prepare a query to get all stop data for the extracted route keys
        stop_query = """
        SELECT * FROM c 
        WHERE ARRAY_CONTAINS(@route_keys, c["route_key"])
        """
        stop_parameters = [{"name": "@route_keys", "value": list(route_keys)}]
        
        # Get stop data
        stop_data = list(stop_container.query_items(
            query=stop_query,
            parameters=stop_parameters,
            enable_cross_partition_query=True
        ))

        return vehicle_data, stop_data

    except exceptions.CosmosHttpResponseError as e:
        print(f"Error querying Cosmos DB: {e}")
        return [], []

In [17]:
# Example Usage
timestamp = "2024-09-21T17:56:00+00:00"  # Replace this with the desired timestamp
utc_diff = 5

# Parse the string into a datetime object
dt_central = datetime.fromisoformat(timestamp)

# Add 5 hours for CDT (UTC-5)
utc_time = dt_central + timedelta(hours=utc_diff)

formatted_iso = utc_time.strftime("%Y-%m-%dT%H:%M:%S") + "+00:00"

print(formatted_iso)

vehicles, stops = get_vehicle_and_stop_data(formatted_iso)

# Display the results
df_buses = pd.DataFrame(vehicles)
df_stops = pd.DataFrame(stops)

# print(df_buses.head())
# print(df_stops.head())

2024-09-21T22:56:00+00:00


In [18]:
route_colors = px.colors.qualitative.Set1

# Create a mapping of route_name to color
route_color_map = {route_name: route_colors[i % len(route_colors)] for i, route_name in enumerate(df_stops['route_name'].unique())}

# Function to create a single combined trace for both points and lines for each route
def create_route_traces(df, color_map):
    traces = []
    route_groups = df.groupby('route_key')

    for route_key, route_data in route_groups:
        # Get the first direction key for this route
        first_direction_key = route_data['direction_key'].iloc[0]
        
        # Filter the route data to include only stops with the same direction_key
        filtered_route_data = route_data[route_data['direction_key'] == first_direction_key]

        if len(filtered_route_data) < 2:
            # Skip if we don't have at least two points to connect
            continue
        
        latitudes = filtered_route_data['latitude'].tolist()
        longitudes = filtered_route_data['longitude'].tolist()
        route_name = filtered_route_data['route_name'].iloc[0]  # Get the route name

        # Append the first point to the end to close the loop
        latitudes.append(latitudes[0])
        longitudes.append(longitudes[0])

        # Create a single trace combining both lines and points for the route
        trace = go.Scattermapbox(
            mode="lines",  # Just lines for the route
            lon=longitudes,
            lat=latitudes,
            line=dict(width=3, color=color_map[route_name]),  # Set line color
            name=route_name,
            legendgroup=route_name,  # Group by route name
            showlegend=True,  # Show in legend
            hoverinfo="text",
            text=[f"Stop: {stop_code}, Route: {route_name}" for stop_code in filtered_route_data['stop_code']]
        )
        traces.append(trace)

        named_stops_data = filtered_route_data[filtered_route_data['stop_name'].notna()]

        if not named_stops_data.empty:
            # Add stops as markers for this route, only for named stops
            trace_stops = go.Scattermapbox(
                mode="markers",
                lon=named_stops_data['longitude'].tolist(),
                lat=named_stops_data['latitude'].tolist(),
                marker=dict(size=8, color=color_map[route_name]),
                name=f"{route_name} Stops",
                legendgroup=route_name,  # Group stops with the route
                showlegend=False,  # Only the route name should show in the legend
                hoverinfo="text",
                text=[f"Stop: {stop_name}, Route: {route_name}" for stop_name in named_stops_data['stop_name']]
            )
            traces.append(trace_stops)

    return traces

# Updated function to create bus traces with larger markers and a black border
def create_bus_traces(df_buses, color_map):
    bus_traces = []
    for _, bus in df_buses.iterrows():
        # Parse amenities, if available
        amenities = bus['amenities'] if pd.notna(bus['amenities']) else "None"
        
        # Parse heading (direction in degrees)
        heading = f"{bus['heading']}°" if pd.notna(bus['heading']) else "N/A"
        
        # Create hover text with more detailed information
        hover_text = (
            f"Bus: {bus['vehicle_name']}<br>"
            f"Route: {bus['route_name']}<br>"
            f"Passengers: {bus['passengers_onboard']}/{bus['passenger_capacity']}<br>"
            f"Amenities: {amenities}<br>"
            f"Speed: {bus['speed']} km/h<br>"
            f"Heading: {heading}<br>"
            f"Last GPS Date: {bus['last_gps_date']}"
        )
        
        # Create the bus trace with larger markers and a black border
        trace = go.Scattermapbox(
            mode="markers",
            lon=[bus['longitude']],
            lat=[bus['latitude']],
            marker=dict(
                size=18,  # Marker size
                color=color_map[bus['route_name']],  # Marker color
                symbol="circle"
                # line=dict(color='black', width=2)  # Black border with 2px width
            ),
            name=f"{bus['route_name']} Buses",
            legendgroup=bus['route_name'],  # Group buses with the route
            showlegend=False,  # Buses are hidden in the legend; only route names are shown
            hoverinfo="text",
            text=hover_text  # Updated hover information
        )
        bus_traces.append(trace)
    return bus_traces

# Create the combined traces (both points and lines) for each route
route_traces = create_route_traces(df_stops, route_color_map)
bus_traces = create_bus_traces(df_buses, route_color_map)

# Initialize a map figure
fig = go.Figure()

# Add the combined traces to the figure
for trace in route_traces:
    fig.add_trace(trace)

# Add the bus traces to the figure
for bus_trace in bus_traces:
    fig.add_trace(bus_trace)
    
fig.update_layout(
    mapbox={
        'style': "open-street-map",
        'center': {'lat': 30.613882, 'lon': -96.350973},
        'zoom': 14
    },
    title="Texas A&M Bus Data at {}".format(dt_central.strftime(f"%A, %B {dt_central.day}, %Y %I:%M:%S %p")),
    margin=dict(l=10, r=10, t=32, b=10),
    legend_tracegroupgap=8
)

fig.show(renderer='iframe')